# W_GL_SEGMENT_D ETL Process
### SCD Type 2 Dimension - Effective Dating with NOT EXISTS Change Detection

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2625027 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS,
  '1899-01-01 00:00:00' AS LOW_DATE, '3714-01-01 00:00:00' AS HI_DT
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Read from staging with aggregation and user lookups
CREATE OR REPLACE TEMP VIEW stg_segment_with_users AS
SELECT
  ds.segment_lov_id, COALESCE(ds.segment_val_code, '__NOT_APPLICABLE__') AS segment_val_code,
  COALESCE(u_created.row_wid, 0) AS created_by_wid,
  COALESCE(u_changed.row_wid, 0) AS changed_by_wid,
  ds.created_on_dt, ds.changed_on_dt,
  ds.aux1_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux2_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux3_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux4_changed_on_dt,
  ds.src_eff_from_dt, ds.src_eff_to_dt,
  CASE WHEN ds.delete_flg = 'Y' THEN 'Y' ELSE 'N' END AS delete_flg,
  ds.datasource_num_id, ds.integration_id,
  CAST(NULL AS STRING) AS set_id,
  ds.tenant_id, ds.x_custom,
  COALESCE(ds.src_eff_from_dt, to_timestamp(p.LOW_DATE, 'yyyy-MM-dd HH:mm:ss')) AS effective_from_dt,
  to_timestamp(p.HI_DT, 'yyyy-MM-dd HH:mm:ss') AS effective_to_dt,
  'Y' AS ind_current_flg
FROM (
  SELECT segment_lov_id, MAX(segment_val_code) AS segment_val_code, MAX(created_by_id) AS created_by_id,
    MAX(changed_by_id) AS changed_by_id, MAX(created_on_dt) AS created_on_dt, MAX(changed_on_dt) AS changed_on_dt,
    MAX(aux1_changed_on_dt) AS aux1_changed_on_dt, src_eff_from_dt, src_eff_to_dt,
    MAX(delete_flg) AS delete_flg, datasource_num_id, integration_id, MAX(tenant_id) AS tenant_id, MAX(x_custom) AS x_custom
  FROM workspace.oracle_edw.w_gl_segment_ds
  GROUP BY segment_lov_id, src_eff_from_dt, src_eff_to_dt, datasource_num_id, integration_id
) ds
LEFT JOIN workspace.oracle_edw.w_user_d u_created
  ON ds.datasource_num_id = u_created.datasource_num_id
  AND ds.created_by_id = u_created.integration_id
  AND ds.changed_on_dt >= u_created.effective_from_dt
  AND ds.changed_on_dt < u_created.effective_to_dt
  AND u_created.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_user_d u_changed
  ON ds.datasource_num_id = u_changed.datasource_num_id
  AND ds.changed_by_id = u_changed.integration_id
  AND ds.changed_on_dt >= u_changed.effective_from_dt
  AND ds.changed_on_dt < u_changed.effective_to_dt
  AND u_changed.delete_flg = 'N'
CROSS JOIN etl_params p;

In [ ]:
%sql
-- 3) CHANGE DETECTION: Find new/changed records using NOT EXISTS
CREATE OR REPLACE TEMP VIEW stg_changes AS
SELECT s.*, 'I' AS ind_update
FROM stg_segment_with_users s
WHERE NOT EXISTS (
  SELECT 1 FROM workspace.oracle_edw.w_gl_segment_d t
  WHERE t.src_eff_from_dt = s.src_eff_from_dt
    AND t.datasource_num_id = s.datasource_num_id
    AND t.integration_id = s.integration_id
    AND COALESCE(t.changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.aux1_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.aux1_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.aux2_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.aux2_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.aux3_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.aux3_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.aux4_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.aux4_changed_on_dt, to_date('3714-01-01', 'yyyy-MM-dd'))
);

In [ ]:
%sql
-- 4) INSERT TYPE 1: Create SCD1_WID entries for new natural keys
INSERT INTO workspace.oracle_edw.w_gl_segment_t1_d (scd1_wid, datasource_num_id, integration_id, w_insert_dt, w_update_dt, etl_proc_wid)
SELECT 
  row_number() OVER (ORDER BY i.datasource_num_id, i.integration_id) + COALESCE((SELECT MAX(scd1_wid) FROM workspace.oracle_edw.w_gl_segment_t1_d), 0),
  i.datasource_num_id, i.integration_id,
  current_timestamp(), current_timestamp(),
  (SELECT ETL_PROC_WID FROM etl_params)
FROM (
  SELECT DISTINCT datasource_num_id, integration_id FROM stg_changes
) i
LEFT JOIN workspace.oracle_edw.w_gl_segment_t1_d t1
  ON t1.datasource_num_id = i.datasource_num_id AND t1.integration_id = i.integration_id
WHERE t1.scd1_wid IS NULL;

In [ ]:
%sql
-- 5) INSERT NEW DIMENSION RECORDS: Insert with SCD1_WID lookup
INSERT INTO workspace.oracle_edw.w_gl_segment_d (
  datasource_num_id, integration_id, segment_lov_id, segment_val_code, set_id, tenant_id, x_custom,
  created_by_wid, changed_by_wid, created_on_dt, changed_on_dt,
  aux1_changed_on_dt, aux2_changed_on_dt, aux3_changed_on_dt, aux4_changed_on_dt,
  src_eff_from_dt, src_eff_to_dt, delete_flg,
  w_insert_dt, w_update_dt, etl_proc_wid, scd1_wid,
  current_flg, effective_from_dt, effective_to_dt
)
SELECT
  s.datasource_num_id, s.integration_id, s.segment_lov_id, s.segment_val_code, s.set_id, s.tenant_id, s.x_custom,
  s.created_by_wid, s.changed_by_wid, s.created_on_dt, s.changed_on_dt,
  s.aux1_changed_on_dt, s.aux2_changed_on_dt, s.aux3_changed_on_dt, s.aux4_changed_on_dt,
  s.src_eff_from_dt, s.src_eff_to_dt, s.delete_flg,
  current_timestamp(), current_timestamp(), (SELECT ETL_PROC_WID FROM etl_params), scd1.scd1_wid,
  s.ind_current_flg, s.effective_from_dt, s.effective_to_dt
FROM stg_changes s
INNER JOIN workspace.oracle_edw.w_gl_segment_t1_d scd1
  ON scd1.datasource_num_id = s.datasource_num_id AND scd1.integration_id = s.integration_id
WHERE s.ind_update = 'I';

In [ ]:
%sql
-- 6) FIX EFFECTIVE DATES: Update effective_to_dt for historized records
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS t
USING (
  SELECT t0.row_wid,
    COALESCE(MIN(i0.effective_from_dt), to_timestamp((SELECT HI_DT FROM etl_params), 'yyyy-MM-dd HH:mm:ss')) AS new_eff_to_dt,
    CASE WHEN MIN(i0.effective_from_dt) IS NULL THEN 'Y' ELSE 'N' END AS new_current_flg
  FROM workspace.oracle_edw.w_gl_segment_d t0
  INNER JOIN workspace.oracle_edw.w_gl_segment_d i0
    ON i0.etl_proc_wid = (SELECT ETL_PROC_WID FROM etl_params)
    AND t0.datasource_num_id = i0.datasource_num_id
    AND t0.integration_id = i0.integration_id
    AND t0.effective_from_dt <= i0.effective_from_dt
    AND t0.effective_to_dt >= i0.effective_from_dt
    AND t0.delete_flg = 'N'
  GROUP BY t0.row_wid
) src
ON t.row_wid = src.row_wid
WHEN MATCHED THEN UPDATE SET
  t.effective_to_dt = src.new_eff_to_dt,
  t.current_flg = src.new_current_flg;

In [ ]:
%sql
-- 7) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLSEGMENTDIMENSION' AS package_name,
    'W_GL_SEGMENT_D' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 8) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLSEGMENTDIMENSION', 'W_GL_SEGMENT_D', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 9) CLEANUP
DROP VIEW IF EXISTS stg_changes;
DROP VIEW IF EXISTS stg_segment_with_users;
DROP VIEW IF EXISTS etl_params;